<a href="https://colab.research.google.com/github/ElsaMolina/AluraStore_Latam/blob/main/AluraStoreLatam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importación de datos



In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda = pd.read_csv(url)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

tienda.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Asistente virtual,Electrónicos,164300.0,6900.0,16/01/2021,Pedro Gomez,Bogotá,4,Tarjeta de crédito,8,4.60971,-74.08175
1,Mesa de comedor,Muebles,192300.0,8400.0,18/05/2022,Beatriz Morales,Medellín,1,Tarjeta de crédito,4,6.25184,-75.56359
2,Juego de mesa,Juguetes,209600.0,15900.0,15/03/2021,Juan Fernandez,Cartagena,1,Tarjeta de crédito,1,10.39972,-75.51444
3,Microondas,Electrodomésticos,757500.0,41000.0,03/05/2022,Juan Fernandez,Cali,4,Nequi,1,3.43722,-76.52250
4,Silla de oficina,Muebles,335200.0,20200.0,07/11/2020,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359


#1. Análisis de facturación



In [14]:
#IngresoT = Ingreso Total
IngresoT_Tienda1 = tienda["Precio"].sum()
IngresoT_Tienda2 = tienda2["Precio"].sum()
IngresoT_Tienda3 = tienda3["Precio"].sum()
IngresoT_Tienda4 = tienda4["Precio"].sum()

print(f"Desglose de los ingresos totales por tienda:")
print(f"Tienda 1: {IngresoT_Tienda1}")
print(f"Tienda 2: {IngresoT_Tienda2}")
print(f"Tienda 3: {IngresoT_Tienda3}")
print(f"Tienda 4: {IngresoT_Tienda4}")


Desglose de los ingresos totales por tienda:
Tienda 1: 1150880400.0
Tienda 2: 1116343500.0
Tienda 3: 1098019600.0
Tienda 4: 1038375700.0


# 2. Ventas por categoría

In [11]:
'''
Creeamos una tabla que muestre los Poductos vendidos por Categoría en cada Tienda,
Ordenando las categorías de mayor a menor cantidad de productos vendidos.
'''
for i,df in enumerate([tienda, tienda2, tienda3, tienda4], 1):
  conteo = df.groupby("Categoría del Producto").size()\
            .reset_index(name=f'Tienda{i}_Cantidad')\
            .sort_values(by=f'Tienda{i}_Cantidad', ascending=False)
  print(f'\nTienda{i} - Productos por categoría:')
  print(conteo)


Tienda1 - Productos por categoría:
    Categoría del Producto  Tienda1_Cantidad
7                  Muebles               465
3             Electrónicos               448
5                 Juguetes               324
2        Electrodomésticos               312
1     Deportes y diversión               284
4   Instrumentos musicales               182
6                   Libros               173
0  Artículos para el hogar               171

Tienda2 - Productos por categoría:
    Categoría del Producto  Tienda2_Cantidad
7                  Muebles               442
3             Electrónicos               422
5                 Juguetes               313
2        Electrodomésticos               305
1     Deportes y diversión               275
4   Instrumentos musicales               224
6                   Libros               197
0  Artículos para el hogar               181

Tienda3 - Productos por categoría:
    Categoría del Producto  Tienda3_Cantidad
7                  Muebles           

# 3. Calificación promedio de la tienda


In [13]:
#Creamos el calculo del promedio de calificación de cada tienda.
calculo_promedio = pd.DataFrame({
    "Tienda": [f'Tienda {i}' for i in range(1, 5)],
    "Calificación promedio": [tienda["Calificación"].mean(),
                              tienda2["Calificación"].mean(),
                              tienda3["Calificación"].mean(),
                              tienda4["Calificación"].mean()]
                                                              })

#Ordenamos de mayor a menor calificación
calculo_promedio = calculo_promedio.sort_values("Calificación promedio", ascending=False).reset_index(drop=True)

print("\nCalificación promedio de cada tienda:")
print(calculo_promedio)


Calificación promedio de cada tienda:
     Tienda  Calificación promedio
0  Tienda 3               4.048326
1  Tienda 2               4.037304
2  Tienda 4               3.995759
3  Tienda 1               3.976685


# 4. Productos más y menos vendidos

In [18]:
'''
Creamos la función para analizar los productos más y menos vendidos en cada tienda
Ordenamos de mayor a menor cantidad de productos vendidos.
'''

def productos_mas_menos_vendidos(tienda, numero_tienda):
  ventas_producto = tienda ["Producto"].value_counts().reset_index()
  ventas_producto.columns = ["Producto", "Cantidad"]

  ventas_producto = ventas_producto.sort_values("Cantidad", ascending=False)

  #Para caluclar el producto más vendido y el producto menos vendido
  mas_vendido = ventas_producto.iloc[0]
  menos_vendido = ventas_producto.iloc[-1]

  print(f'\nAnálisis Tienda {numero_tienda}:')
  print(f'Producto más vendido: {mas_vendido ["Producto"]} con {mas_vendido["Cantidad"]} unidades')
  print(f'Producto más vendido: {menos_vendido ["Producto"]} con {menos_vendido ["Cantidad"]} unidades')

  return ventas_producto

#Ahora aplicamos la función los datos
resultados ={}
for i, df in enumerate([tienda, tienda2, tienda3, tienda4], 1):
  resultados[f'Tienda {i}'] = productos_mas_menos_vendidos(df, i)


Análisis Tienda 1:
Producto más vendido: Microondas con 60 unidades
Producto más vendido: Celular ABXY con 33 unidades

Análisis Tienda 2:
Producto más vendido: Iniciando en programación con 65 unidades
Producto más vendido: Juego de mesa con 32 unidades

Análisis Tienda 3:
Producto más vendido: Kit de bancas con 57 unidades
Producto más vendido: Bloques de construcción con 35 unidades

Análisis Tienda 4:
Producto más vendido: Cama box con 62 unidades
Producto más vendido: Guitarra eléctrica con 33 unidades


In [46]:
from IPython.display import display, HTML

# Crear DataFrame consolidado
ventas_tiendas = pd.concat([
    tienda.assign(Tienda='Tienda 1'),
    tienda2.assign(Tienda='Tienda 2'),
    tienda3.assign(Tienda='Tienda 3'),
    tienda4.assign(Tienda='Tienda 4')
])

# Crear tabla pivot
tabla_ventas = ventas_tiendas.pivot_table(
    index=['Producto', 'Categoría del Producto'],
    columns='Tienda',
    values='Cantidad_Vendida',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Mostrar tabla con estilo
display(HTML(
    tabla_ventas.style
    .set_caption('Ventas por Producto en Cada Tienda')
    .set_properties(**{'text-align': 'center'})
    .background_gradient(cmap='Blues', subset=['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4'])
    .format('{:.0f}', subset=['Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 4'])
    .hide(axis='index')
    .to_html()
))

KeyError: 'Cantidad_Vendida'

# 5. Envío promedio por tienda